## Load Libraries

In [1]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
import pickle
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

import matplotlib.font_manager
from matplotlib import style
style.use('seaborn') or plt.style.use('seaborn')

from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
import plotly.graph_objects as go
import seaborn as sns

from collections import Counter
import operator


from xgboost import plot_importance


from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

from sklearn.decomposition import FactorAnalysis

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss


from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet
import math

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle


import scipy
from sklearn import metrics


from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [2]:
import my_bib as mb

## Define functions

In [3]:
def get_color(var, X_cat_, color_dict_):

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    index_code = X_name.index(var)
    cat = X_factors[index_code]

    return color_dict_[cat]

color_dict = {}
color_dict['Country'] = '#484849'
color_dict['HF'] = '#A31300'
color_dict['LSF'] = '#FF9E0D'
color_dict['PSF'] = '#FF4800'
color_dict['SDHF'] = '#1C4F9E'
color_dict['DF'] = '#009E32'

def get_bar_colors(data_, X_cat_):
    color_dict = {}
    color_dict['Country'] = '#484849'
    color_dict['HF'] = '#A31300'
    color_dict['LSF'] = '#FF9E0D'
    color_dict['PSF'] = '#FF4800'
    color_dict['SDHF'] = '#1C4F9E'
    color_dict['DF'] = '#009E32'
    
    #color_dict = {}
    #color_dict = {'DF':'#5975a4', 'MF':'#cc8963', 'SF':'#5f9e6e', 'SLF':'#b55d60',
    #              'n':'#857aab', 'country':'#8d7866'}#, '#d095bf'}

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    bar_color_total = []
    for i in list(data_.Features):
        if(i == 'Country'):
            bar_color_total.append(color_dict['Country'])
            continue
        index_code = X_name.index(i)
        cat = X_factors[index_code]
        bar_color_total.append(color_dict[cat])
    return bar_color_total


def plot_estimate_value(regression_model, X_cat_ = [], title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, pvalue_type = 'False'):

    df = regression_model[0]
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()
    data = df.sort_values('Estimate', ascending=False)

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, color = 'darkblue')
        plt.xlim(xlim)


    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        if(pvalue_type == 'color'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                                 '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],2)),
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'white',
                                 bbox=dict(boxstyle="round",
                                           ec=color,
                                            fc=color,
                                           )
                                 )
                y_step+=1

        elif(pvalue_type == 'value'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                 '(' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],8))+')',
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'black',

                                 )
                y_step+=1
        else:
            if(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.01):
                
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):
                  
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )      
            y_step+=1
            
         
    text_diff =xlim[1]/2.2
    plt.text(xlim[1] - text_diff, df.shape[0]-1.5,r'$ R^2 $(' + str(np.round(regression_model[1],2)) + ') \t$F^2 $(' + str(np.round(regression_model[3],2))+ ') \n$MSE $(' + str(np.round(regression_model[4],2))+ ') \t$RMSE $(' + str(np.round(regression_model[5],2)) + ')',
                             size= 12, rotation=0.,
                             ha="left", va="center", color = 'black',
                             bbox=dict(boxstyle="round",
                                       ec='gray',
                                        fc='gray',
                                       )
                             )


    plt.locator_params(axis='x', nbins=4)

    
    
def plot_estimate_value_no_sort(regression_model, title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, ylabel = True, ylabelR = False ):

    df = regression_model
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()

    #plt.figure(figsize=(8,12))
    plt.title(title)
    sns.barplot(x="Estimate", y="Features", data = df, color = 'Brown')
    if(ylabel == False):
        plt.ylabel('')
        plt.yticks([])
    plt.xlim(xlim)
    
    if(ylabelR):
        plt.tick_params (axis = 'y', which = 'both', labelleft = False, labelright = True)

    y_step=0  
    for i in range(df.shape[0]):
        
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'yellow'
        else:
            color = 'red'   
            
        plt.text(df['Estimate'].iloc[y_step]+0.015, y_step, 
                         '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],5)),
                         size= size, rotation=0.,
                         ha="left", va="center", color = color,
                         bbox=dict(boxstyle="round",
                                   ec=(50/255, 50/255, 50/255),
                                    fc=(50/255, 50/255, 50/255),
                                   )
                         )
        y_step+=1
        

def conf_interval(database):
    r_squared_list = []

    y_database = database['MMSE']
    X_database = database.drop('MMSE', axis=1)

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
            Ridge(),
            {
                'alpha': ( 0.0001, 0.01, 0.001),
                'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                'max_iter': (1000, 10000, 100000, 1000000),
            },
            n_iter=10,
            random_state=i, 
            scoring='r2',
            cv=3
        )

        opt_Ridge.fit(X_train, y_train)

        r_squared_list.append(opt_Ridge.score(X_test, y_test))


    print(r'99% confidence interval (+-', np.round(np.std(r_squared_list)*2.58, 4),')')
    return np.round(np.std(r_squared_list)*2.58, 4)


def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_tval_XGB_tree(clf, X, y):
    a = np.nan
    b = np.array(clf.feature_importances_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def coef_pval_XGB_tree(clf, X, y):

    n = X.shape[0]
    t = coef_tval_XGB_tree(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)



def summary(clf, X, y, xlabels=None, regressor = ''):

    print('Resumen del regresor ' + regressor + '\n')
    
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')

    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate','t value', 'p value']
    )
    
    if(regressor == 'XGBRegressor'):
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_[0]]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
    elif(regressor == 'XGBRegressorNoLinear'):
        coef_df = pd.DataFrame(
            index=['_intercept'] + list(xlabels),
            columns=['Estimate','t value', 'p value']
        )

        coef_df['Estimate'] = np.concatenate(
                (np.round(np.array([np.nan]), 6), np.round((clf.feature_importances_), 6)))

        coef_df['t value'] = np.round(coef_tval_XGB_tree(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval_XGB_tree(clf, X, y), 20)
            # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
                'Min': pd.Series(np.round(resids.min(), 4)),
                '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
                'Median': pd.Series(np.round(np.median(resids), 4)),
                '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
                'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
            # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)

        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
               r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
                f_sq))
    else:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        mse_ = np.round(mse(clf, X, y), 6)
        rmse_ = np.round(rmse(clf, X, y), 6)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
        print('MSE:  {0:.5f}'.format(
            mse_))        
        print('RMSE:  {0:.5f}'.format(
            rmse_))        
    print('---------------------------------------------------------------------------\n\n')
    
    
    
    empty_str = []
    for i in range(coef_df.shape[0]):
        empty_str.append('')
    
    coef_df['value'] = empty_str
    
    coef_df = coef_df.T
    coef_df['R-squared'] = ['','','', r_sq]
    coef_df['Adjusted R-squared'] = ['','','', r_sq_adj]
    coef_df['F-squared'] = ['','','', f_sq]
    #coef_df['MSE'] = ['','','', mse_]
    #coef_df['RMSE'] = ['','','', rmse_]
    return [coef_df.T, r_sq, r_sq_adj, f_sq, mse_, rmse_]

## Load Data

In [4]:
sabe_chile = pd.read_excel('../Data/brazil/SABE_chile.xlsx')
sabe_chile = sabe_chile.iloc[:,1::]

sabe_uruguay = pd.read_excel('../Data/brazil/SABE_uruguay.xlsx')
sabe_uruguay = sabe_uruguay.iloc[:,1::]

sabe_ecuador = pd.read_excel('../Data/brazil/SABE_ecuador.xlsx')
sabe_ecuador = sabe_ecuador.iloc[:,1::]

sabe_colombia = pd.read_excel('../Data/brazil/SABE_colombia.xlsx')
sabe_colombia = sabe_colombia.iloc[:,1::]

sabe_brazil = pd.read_excel('../Data/brazil/SABE_brazil_full.xlsx')
sabe_brazil = sabe_brazil.iloc[:,1::]

/home/marcoantonio/miniconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Add colors for future use, remove non-common features, order all df vars in same order

In [5]:
X_chile_01 = sabe_chile.copy()
X_uruguay_01 = sabe_uruguay.copy()
X_ecuador_01 = sabe_ecuador.copy()
X_colombia_01 = sabe_colombia.copy()
X_brazil_01 = sabe_brazil.copy()

X_cat = pd.read_csv('../Data/brazil/var_name_color_code_new.csv', encoding='latin-1', sep=";")

In [6]:
order_col = list(X_chile_01.columns)
order_col.remove('FS_Adversidad_AgresionVerb_j28')
order_col.remove('FS_Adversidad_AgresionFis_j27')
order_col.remove('FM_CardioMetab_IMC_none')
order_col.remove('FS_EstSocEcon_ElectricEquipment_none')
order_col.remove('FM_CardioMetab_ACV_c09')

X_chile_01     =  X_chile_01[order_col]
X_uruguay_01   =  X_uruguay_01[order_col]
X_ecuador_01   =  X_ecuador_01[order_col]
X_colombia_01  =  X_colombia_01[order_col]
X_brazil_01  = X_brazil_01[order_col]

X_chile_01.columns =  X_cat.newname
X_uruguay_01.columns =  X_cat.newname
X_ecuador_01.columns =  X_cat.newname
X_colombia_01.columns =  X_cat.newname
X_brazil_01.columns = X_cat.newname

In [7]:
print(X_chile_01.shape, '\n', X_uruguay_01.shape, '\n', X_ecuador_01.shape, '\n', 
      X_colombia_01.shape, '\n', X_brazil_01.shape)

(1301, 15) 
 (1450, 15) 
 (5235, 15) 
 (23694, 15) 
 (9412, 15)


In [8]:
#Sanity check
for i in range(len(order_col)):
    print('', X_chile_01.columns[i], '\n', X_uruguay_01.columns[i], '\n', X_ecuador_01.columns[i], '\n', 
          X_colombia_01.columns[i], '\n', X_brazil_01.columns[i])
    print('-------------------------------------------')

 Age 
 Age 
 Age 
 Age 
 Age
-------------------------------------------
 Sex 
 Sex 
 Sex 
 Sex 
 Sex
-------------------------------------------
 Diabetes 
 Diabetes 
 Diabetes 
 Diabetes 
 Diabetes
-------------------------------------------
 Education 
 Education 
 Education 
 Education 
 Education
-------------------------------------------
 Live Alone 
 Live Alone 
 Live Alone 
 Live Alone 
 Live Alone
-------------------------------------------
 House Condition 
 House Condition 
 House Condition 
 House Condition 
 House Condition
-------------------------------------------
 Hypertension 
 Hypertension 
 Hypertension 
 Hypertension 
 Hypertension
-------------------------------------------
 Heart Disease 
 Heart Disease 
 Heart Disease 
 Heart Disease 
 Heart Disease
-------------------------------------------
 Alcohol consumption 
 Alcohol consumption 
 Alcohol consumption 
 Alcohol consumption 
 Alcohol consumption
-------------------------------------------
 Physical activity

In [9]:
len(X_brazil_01.columns)

15

In [10]:
parameter_dict = {}

## Prepare data

### Chile

In [11]:
X_chile_n_IMC_ACV = X_chile_01.drop(['Barthel' ], axis=1)

In [12]:
X_chile_n_IMC_ACV.dropna(inplace=True)
print(X_chile_n_IMC_ACV.shape, X_chile_01.shape, X_chile_01.shape[0] - X_chile_n_IMC_ACV.shape[0])

(1072, 14) (1301, 15) 229


In [13]:
y_chile = X_chile_n_IMC_ACV['MMSE']
X_chile = X_chile_n_IMC_ACV.drop('MMSE', axis=1)

### Uruguay

In [14]:
X_uruguay_n_IMC_ACV = X_uruguay_01.drop(['Barthel' ], axis=1)

In [15]:
X_uruguay_n_IMC_ACV.dropna(inplace=True)
print(X_uruguay_n_IMC_ACV.shape, X_uruguay_01.shape, X_uruguay_01.shape[0] - X_uruguay_n_IMC_ACV.shape[0])

(1268, 14) (1450, 15) 182


In [16]:
y_uruguay = X_uruguay_n_IMC_ACV['MMSE']
X_uruguay = X_uruguay_n_IMC_ACV.drop('MMSE', axis=1)

### Ecuador

In [17]:
X_ecuador_n_IMC_ACV = X_ecuador_01.drop(['Barthel' ], axis=1)

In [18]:
X_ecuador_n_IMC_ACV.dropna(inplace=True)
print(X_ecuador_n_IMC_ACV.shape, X_ecuador_01.shape, X_ecuador_01.shape[0] - X_ecuador_n_IMC_ACV.shape[0])

(3021, 14) (5235, 15) 2214


In [19]:
y_ecuador = X_ecuador_n_IMC_ACV['MMSE']
X_ecuador = X_ecuador_n_IMC_ACV.drop('MMSE', axis=1)

### Colombia

In [20]:
X_colombia_n_IMC_ACV = X_colombia_01.drop(['Barthel' ], axis=1)


In [21]:
X_colombia_n_IMC_ACV.dropna(inplace=True)
print(X_colombia_n_IMC_ACV.shape, X_colombia_01.shape, X_colombia_01.shape[0] - X_colombia_n_IMC_ACV.shape[0])

(22748, 14) (23694, 15) 946


In [22]:
y_colombia = X_colombia_n_IMC_ACV['MMSE']
X_colombia = X_colombia_n_IMC_ACV.drop('MMSE', axis=1)

### Brazil

In [23]:
X_brazil_n_IMC_ACV = X_brazil_01.drop(['Barthel' ], axis=1)

In [24]:
X_brazil_n_IMC_ACV.dropna(inplace=True)
print(X_brazil_n_IMC_ACV.shape, X_brazil_01.shape, X_brazil_01.shape[0] - X_brazil_n_IMC_ACV.shape[0])

(7844, 14) (9412, 15) 1568


In [25]:
y_brazil = X_brazil_n_IMC_ACV['MMSE']
X_brazil = X_brazil_n_IMC_ACV.drop('MMSE', axis=1)

## Results

### HIC: Chile and Uruguay joint data


In [26]:
X_chile_n_IMC_ACV_colEcua = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua = X_uruguay_n_IMC_ACV.copy()

In [27]:
X_chile_n_IMC_ACV_colEcua['Country'] = np.ones([X_chile_n_IMC_ACV_colEcua.shape[0]])
X_uruguay_n_IMC_ACV_colEcua['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua.shape[0]])*2

In [28]:
X_chile_uruguay_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua, X_uruguay_n_IMC_ACV_colEcua], axis=0,))

In [29]:
y_chile_uruguay = X_chile_uruguay_n_IMC_ACV['MMSE']
X_chile_uruguay = X_chile_uruguay_n_IMC_ACV.drop('MMSE', axis=1)

In [30]:
print(X_chile_uruguay.shape)

(2340, 14)


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_chile_uruguay, y_chile_uruguay, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')



Ridge
val. score: 0.10238689312313827
test score: 0.09576470049354191
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 10000), ('solver', 'sparse_cg')])
---------------------------------------------



In [32]:
modelRidge_chile_uruguay = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_chile_uruguay.fit(X_chile_uruguay, y_chile_uruguay)

parameter_dict['chile_uruguay1'] = opt_Ridge.best_params_

modelRidge_sum_chile_uruguay = summary(modelRidge_chile_uruguay, X_chile_uruguay, y_chile_uruguay, X_chile_uruguay.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-1.6111 -0.5091 -0.1732 0.2831 8.8306


Coefficients:
                     Estimate  t value       p value
_intercept           1.329771   3.3930  7.029254e-04
Age                 -0.033980 -58.8696  0.000000e+00
Sex                 -0.115723  -2.8383  4.574854e-03
Diabetes             0.014626   0.2835  7.768082e-01
Education            0.095167   2.6704  7.628151e-03
Live Alone           0.072168   1.5138  1.302093e-01
House Condition      0.102107   5.2844  1.378280e-07
Hypertension         0.001774   0.0491  9.608336e-01
Heart Disease       -0.027107  -0.6718  5.017549e-01
Alcohol consumption  0.044836   2.4676  1.367376e-02
Physical activity   -0.115930  -2.6119  9.060619e-03
Smoking status      -0.060233  -1.5856  1.129740e-01
Falls                0.124335   3.2181  1.307936e-03
Mental Problems      0.170245   3.7537  1.785222e-04
Country             -0.095105  -2.6285  8.632593e-03
---
R-squared:  0.109

### HIC: Chile, Uruguay and Brazil joint data


In [33]:
X_chile_n_IMC_ACV_colEcua = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua = X_uruguay_n_IMC_ACV.copy()
X_brazil_n_IMC_ACV_colEcua = X_brazil_n_IMC_ACV.copy()

In [34]:
X_chile_n_IMC_ACV_colEcua['Country'] = np.ones([X_chile_n_IMC_ACV_colEcua.shape[0]])
X_uruguay_n_IMC_ACV_colEcua['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua.shape[0]])
X_brazil_n_IMC_ACV_colEcua['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua.shape[0]])*2

In [35]:
X_chi_uru_br_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua, X_uruguay_n_IMC_ACV_colEcua, X_brazil_n_IMC_ACV_colEcua], axis=0,))

In [36]:
y_chile_uruguay_br = X_chi_uru_br_n_IMC_ACV['MMSE']
X_chile_uruguay_br = X_chi_uru_br_n_IMC_ACV.drop('MMSE', axis=1)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_chile_uruguay_br, y_chile_uruguay_br, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')



Ridge
val. score: 0.19946189546292306
test score: 0.20996474566406087
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 10000), ('solver', 'lsqr')])
---------------------------------------------



In [38]:
modelRidge_chile_uruguay_br = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_chile_uruguay_br.fit(X_chile_uruguay_br, y_chile_uruguay_br)

parameter_dict['chile_uruguay_br'] = opt_Ridge.best_params_

modelRidge_sum_chile_uruguay_br = summary(modelRidge_chile_uruguay_br, X_chile_uruguay_br, y_chile_uruguay_br, X_chile_uruguay_br.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.1827 -0.4449 -0.0745 0.3247 8.9268


Coefficients:
                     Estimate   t value       p value
_intercept           1.170149    9.6690  0.000000e+00
Age                 -0.023807 -146.0884  0.000000e+00
Sex                 -0.018316   -1.3089  1.905987e-01
Diabetes             0.012508    0.6935  4.879904e-01
Education            0.269897   22.1724  0.000000e+00
Live Alone          -0.027332   -1.4829  1.381205e-01
House Condition      0.033258   15.5181  0.000000e+00
Hypertension         0.000952    0.0698  9.443201e-01
Heart Disease       -0.025587   -1.3303  1.834425e-01
Alcohol consumption  0.023656    2.1754  2.961998e-02
Physical activity   -0.104159   -7.4291  1.181277e-13
Smoking status      -0.052055   -3.6805  2.339413e-04
Falls                0.045454    2.9387  3.302825e-03
Mental Problems      0.157516   10.4774  0.000000e+00
Country             -0.103998   -2.9250  3.452160e-03
---
R

### LMIC: Ecuador, Colombia and Brazil joint data

In [39]:
X_ecuador_n_IMC_ACV_colEcua = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua = X_colombia_n_IMC_ACV.copy()
X_brazil_n_IMC_ACV_colEcua = X_brazil_n_IMC_ACV.copy()

In [40]:
X_ecuador_n_IMC_ACV_colEcua['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])
X_colombia_n_IMC_ACV_colEcua['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])
X_brazil_n_IMC_ACV_colEcua['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua.shape[0]])*2

In [41]:
X_ecu_col_br_n_IMC_ACV = shuffle(pd.concat([X_ecuador_n_IMC_ACV_colEcua, X_colombia_n_IMC_ACV_colEcua, X_brazil_n_IMC_ACV_colEcua], axis=0,))

In [42]:
y_ecu_col_br = X_ecu_col_br_n_IMC_ACV['MMSE']
X_ecu_col_br = X_ecu_col_br_n_IMC_ACV.drop('MMSE', axis=1)

In [43]:
print(X_ecu_col_br.shape)

(33613, 14)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_ecu_col_br, y_ecu_col_br, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.21741671270062482
test score: 0.20686777402982104
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 1000), ('solver', 'sag')])
---------------------------------------------



In [45]:
modelRidge_ecu_col_br = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_ecu_col_br.fit(X_ecu_col_br, y_ecu_col_br)

parameter_dict['ecu_col_br'] = opt_Ridge.best_params_

modelRidge_sum_ecu_col_br = summary(modelRidge_ecu_col_br, X_ecu_col_br, y_ecu_col_br, X_ecu_col_br.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.4144 -0.5278 -0.1476 0.3396 5.1359


Coefficients:
                     Estimate   t value       p value
_intercept           2.410713   30.0958  0.000000e+00
Age                 -0.041968 -395.4625  0.000000e+00
Sex                 -0.007969   -0.8430  3.992477e-01
Diabetes            -0.001200   -0.1015  9.191561e-01
Education            0.178839   18.3009  0.000000e+00
Live Alone           0.105734    9.5913  0.000000e+00
House Condition      0.043644   25.7995  0.000000e+00
Hypertension         0.000494    0.0543  9.567360e-01
Heart Disease       -0.021894   -1.6879  9.143586e-02
Alcohol consumption  0.044623    4.4788  7.530682e-06
Physical activity   -0.120126  -11.6248  0.000000e+00
Smoking status      -0.033303   -3.9859  6.735328e-05
Falls                0.067334    6.9902  2.795986e-12
Mental Problems      0.231734   18.1131  0.000000e+00
Country             -0.309744  -15.5664  0.000000e+00
---
R

### Ecuador and Colombia joint data

In [46]:
X_ecuador_n_IMC_ACV_colEcua = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua = X_colombia_n_IMC_ACV.copy()

In [47]:
X_ecuador_n_IMC_ACV_colEcua['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])
X_colombia_n_IMC_ACV_colEcua['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2

In [48]:
X_ecuador_colombia_n_IMC_ACV = shuffle(pd.concat([X_ecuador_n_IMC_ACV_colEcua, X_colombia_n_IMC_ACV_colEcua], axis=0,))

In [49]:
y_ecuador_colombia = X_ecuador_colombia_n_IMC_ACV['MMSE']
X_ecuador_colombia = X_ecuador_colombia_n_IMC_ACV.drop('MMSE', axis=1)

In [50]:
print(X_ecuador_colombia.shape)

(25769, 14)


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_ecuador_colombia, y_ecuador_colombia, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.22607074287753917
test score: 0.23307227777133965
best parameters: OrderedDict([('alpha', 0.01), ('max_iter', 100000), ('solver', 'lsqr')])
---------------------------------------------



In [52]:
modelRidge_ecuador_colombia = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_ecuador_colombia.fit(X_ecuador_colombia, y_ecuador_colombia)

parameter_dict['ecuador_colombia'] = opt_Ridge.best_params_

modelRidge_sum_ecuador_colombia = summary(modelRidge_ecuador_colombia, X_ecuador_colombia, y_ecuador_colombia, X_ecuador_colombia.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.3129 -0.5385 -0.1634 0.3262 5.0521


Coefficients:
                     Estimate   t value       p value
_intercept           2.446715   23.5492  0.000000e+00
Age                 -0.050509 -378.6932  0.000000e+00
Sex                  0.008697    0.7446  4.565076e-01
Diabetes             0.005364    0.3752  7.075040e-01
Education            0.117545    8.8916  0.000000e+00
Live Alone           0.174742   10.9911  0.000000e+00
House Condition      0.050577   22.3607  0.000000e+00
Hypertension         0.000334    0.0305  9.757068e-01
Heart Disease       -0.026295   -1.7151  8.634756e-02
Alcohol consumption  0.036820    2.9791  2.893840e-03
Physical activity   -0.122044   -9.1346  0.000000e+00
Smoking status      -0.019013   -1.7359  8.259795e-02
Falls                0.078225    6.8859  5.872858e-12
Mental Problems      0.280519   15.5037  0.000000e+00
Country              0.025860    1.3538  1.758108e-01
---
R

### Brazil

In [53]:
X_brazil_n_IMC_ACV_colEcua = X_brazil_n_IMC_ACV.copy()

# X_brazil_n_IMC_ACV_colEcua['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua.shape[0]])

X_br_n_IMC_ACV = shuffle(pd.concat([X_brazil_n_IMC_ACV_colEcua], axis=0,))

y_br = X_br_n_IMC_ACV['MMSE']
X_br = X_br_n_IMC_ACV.drop('MMSE', axis=1)

print(X_br.shape)

(7844, 13)


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_br, y_br, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')

Ridge
val. score: 0.25965485075275235
test score: 0.2559548765912323
best parameters: OrderedDict([('alpha', 0.0001), ('max_iter', 100000), ('solver', 'lsqr')])
---------------------------------------------



In [55]:
modelRidge_br = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_br.fit(X_br, y_br)

parameter_dict['br'] = opt_Ridge.best_params_

modelRidge_sum_br = summary(modelRidge_br, X_br, y_br, X_br.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.1407 -0.3989 -0.0488 0.3391 2.9491


Coefficients:
                     Estimate   t value       p value
_intercept           0.766141    7.5329  5.506706e-14
Age                 -0.021432 -131.6803  0.000000e+00
Sex                  0.011312    0.7954  4.264305e-01
Diabetes             0.011521    0.6365  5.244680e-01
Education            0.321097   26.0565  0.000000e+00
Live Alone          -0.037272   -1.9179  5.515515e-02
House Condition      0.028304   12.2726  0.000000e+00
Hypertension         0.005474    0.3917  6.953195e-01
Heart Disease       -0.015960   -0.7410  4.587352e-01
Alcohol consumption  0.028245    1.2046  2.284108e-01
Physical activity   -0.104373   -7.4948  7.371881e-14
Smoking status      -0.055687   -2.9817  2.874881e-03
Falls                0.016530    1.0190  3.082394e-01
Mental Problems      0.159552   10.6446  0.000000e+00
---
R-squared:  0.26444,    Adjusted R-squared:  0.26321
F-

### All LAC1: Chile, Uruguay, Ecuador, Colombia, Brazil

In [56]:
X_chile_n_IMC_ACV_colEcua_01 = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua_01 = X_uruguay_n_IMC_ACV.copy()
X_ecuador_n_IMC_ACV_colEcua_01 = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua_01 = X_colombia_n_IMC_ACV.copy()
X_brazil_n_IMC_ACV_colEcua_01 = X_brazil_n_IMC_ACV.copy()

In [57]:
X_chile_n_IMC_ACV_colEcua_01['Country']   = np.ones([X_chile_n_IMC_ACV_colEcua_01.shape[0]])
X_uruguay_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua_01.shape[0]])
X_ecuador_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])*2
X_colombia_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2
X_brazil_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua_01.shape[0]])*2

In [58]:
X_all_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua_01, X_uruguay_n_IMC_ACV_colEcua_01, X_ecuador_n_IMC_ACV_colEcua_01, X_colombia_n_IMC_ACV_colEcua_01, X_brazil_n_IMC_ACV_colEcua_01], axis=0,))

In [59]:
y_all = X_all_n_IMC_ACV['MMSE']
X_all = X_all_n_IMC_ACV.drop('MMSE', axis=1)

In [60]:
print(X_all.shape)

(35953, 14)


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.20123044382073332
test score: 0.20976167337394092
best parameters: OrderedDict([('alpha', 0.01), ('max_iter', 10000), ('solver', 'sag')])
---------------------------------------------



In [62]:
modelRidge_all = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_all.fit(X_all, y_all)

parameter_dict['all'] = opt_Ridge.best_params_

modelRidge_sum_all = summary(modelRidge_all, X_all, y_all, X_all.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.3524 -0.5315 -0.1529 0.3367 8.8431


Coefficients:
                     Estimate   t value       p value
_intercept           1.515238   21.1142  0.000000e+00
Age                 -0.040213 -379.4936  0.000000e+00
Sex                 -0.002294   -0.2483  8.038821e-01
Diabetes            -0.000034   -0.0030  9.976424e-01
Education            0.151033   16.2687  0.000000e+00
Live Alone           0.044847    4.5418  5.596144e-06
House Condition      0.050787   31.2410  0.000000e+00
Hypertension         0.004823    0.5458  5.851818e-01
Heart Disease       -0.029693   -2.4050  1.617666e-02
Alcohol consumption  0.099514   12.6577  0.000000e+00
Physical activity   -0.105703  -10.4633  0.000000e+00
Smoking status       0.040069    5.3949  6.898588e-08
Falls                0.065761    7.0523  1.791234e-12
Mental Problems      0.247494   19.9777  0.000000e+00
Country              0.136224    7.9943  1.332270e-15
---
R

### All LAC2: Chile, Uruguay, Ecuador, Colombia

In [63]:
X_chile_n_IMC_ACV_colEcua_01 = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua_01 = X_uruguay_n_IMC_ACV.copy()
X_ecuador_n_IMC_ACV_colEcua_01 = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua_01 = X_colombia_n_IMC_ACV.copy()

In [64]:
X_chile_n_IMC_ACV_colEcua_01['Country']   = np.ones([X_chile_n_IMC_ACV_colEcua_01.shape[0]])
X_uruguay_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua_01.shape[0]])
X_ecuador_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])*2
X_colombia_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2

In [65]:
X_all_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua_01, X_uruguay_n_IMC_ACV_colEcua_01, X_ecuador_n_IMC_ACV_colEcua_01, X_colombia_n_IMC_ACV_colEcua_01], axis=0,))

In [66]:
y_all = X_all_n_IMC_ACV['MMSE']
X_all = X_all_n_IMC_ACV.drop('MMSE', axis=1)

In [67]:
print(X_all.shape)

(28109, 14)


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.21464317565900934
test score: 0.22177192633539056
best parameters: OrderedDict([('alpha', 0.01), ('max_iter', 10000), ('solver', 'sparse_cg')])
---------------------------------------------



In [69]:
modelRidge_all2 = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_all2.fit(X_all, y_all)

parameter_dict['all'] = opt_Ridge.best_params_

modelRidge_sum_all2 = summary(modelRidge_all2, X_all, y_all, X_all.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.2537 -0.5376 -0.1672 0.3221 8.8832


Coefficients:
                     Estimate   t value       p value
_intercept           2.287234   24.6842  0.000000e+00
Age                 -0.049336 -384.0320  0.000000e+00
Sex                 -0.000997   -0.0888  9.292425e-01
Diabetes             0.007378    0.5371  5.912168e-01
Education            0.116229    9.3931  0.000000e+00
Live Alone           0.156055   11.9388  0.000000e+00
House Condition      0.050493   23.6336  0.000000e+00
Hypertension         0.001827    0.1742  8.617255e-01
Heart Disease       -0.026975   -1.8787  6.029572e-02
Alcohol consumption  0.033135    3.3457  8.219591e-04
Physical activity   -0.124179   -9.8056  0.000000e+00
Smoking status      -0.021489   -2.0414  4.122505e-02
Falls                0.081356    7.4795  7.682743e-14
Mental Problems      0.264985   15.7943  0.000000e+00
Country              0.097372    5.2681  1.388546e-07
---
R

## Summary Results



### HIC



In [72]:
#modelRidge_sum_chile_uruguay[0].to_csv('./res_files/Cross_cog/Ridge_HIC.csv')
modelRidge_sum_chile_uruguay[0]

,Estimate,t value,p value,value
_intercept,1.329771,3.393,0.000703,
Age,-0.03398,-58.8696,0.0,
Sex,-0.115723,-2.8383,0.004575,
Diabetes,0.014626,0.2835,0.776808,
Education,0.095167,2.6704,0.007628,
Live Alone,0.072168,1.5138,0.130209,
House Condition,0.102107,5.2844,0.0,
Hypertension,0.001774,0.0491,0.960834,
Heart Disease,-0.027107,-0.6718,0.501755,
Alcohol consumption,0.044836,2.4676,0.013674,


In [73]:
#modelRidge_sum_chile_uruguay_br[0].to_csv('./res_files/Cross_cog/Ridge_HIC.csv')
modelRidge_sum_chile_uruguay_br[0]

,Estimate,t value,p value,value
_intercept,1.170149,9.669,0.0,
Age,-0.023807,-146.0884,0.0,
Sex,-0.018316,-1.3089,0.190599,
Diabetes,0.012508,0.6935,0.48799,
Education,0.269897,22.1724,0.0,
Live Alone,-0.027332,-1.4829,0.13812,
House Condition,0.033258,15.5181,0.0,
Hypertension,0.000952,0.0698,0.94432,
Heart Disease,-0.025587,-1.3303,0.183442,
Alcohol consumption,0.023656,2.1754,0.02962,


### LMIC

In [74]:
#modelRidge_sum_ecu_col_br[0].to_csv('./res_files/Cross_cog/Ridge_LMIC.csv')
modelRidge_sum_ecu_col_br[0]

,Estimate,t value,p value,value
_intercept,2.410713,30.0958,0.0,
Age,-0.041968,-395.4625,0.0,
Sex,-0.007969,-0.843,0.399248,
Diabetes,-0.0012,-0.1015,0.919156,
Education,0.178839,18.3009,0.0,
Live Alone,0.105734,9.5913,0.0,
House Condition,0.043644,25.7995,0.0,
Hypertension,0.000494,0.0543,0.956736,
Heart Disease,-0.021894,-1.6879,0.091436,
Alcohol consumption,0.044623,4.4788,0.000008,


In [76]:
#modelRidge_sum_ecu_col_br[0].to_csv('./res_files/Cross_cog/Ridge_LMIC.csv')
modelRidge_sum_ecuador_colombia[0]

,Estimate,t value,p value,value
_intercept,2.446715,23.5492,0.0,
Age,-0.050509,-378.6932,0.0,
Sex,0.008697,0.7446,0.456508,
Diabetes,0.005364,0.3752,0.707504,
Education,0.117545,8.8916,0.0,
Live Alone,0.174742,10.9911,0.0,
House Condition,0.050577,22.3607,0.0,
Hypertension,0.000334,0.0305,0.975707,
Heart Disease,-0.026295,-1.7151,0.086348,
Alcohol consumption,0.03682,2.9791,0.002894,


### All LAC

In [75]:
#modelRidge_sum_all[0].to_csv('./res_files/Cross_cog/Ridge_All.csv')
modelRidge_sum_all[0]

,Estimate,t value,p value,value
_intercept,1.515238,21.1142,0.0,
Age,-0.040213,-379.4936,0.0,
Sex,-0.002294,-0.2483,0.803882,
Diabetes,-0.000034,-0.003,0.997642,
Education,0.151033,16.2687,0.0,
Live Alone,0.044847,4.5418,0.000006,
House Condition,0.050787,31.241,0.0,
Hypertension,0.004823,0.5458,0.585182,
Heart Disease,-0.029693,-2.405,0.016177,
Alcohol consumption,0.099514,12.6577,0.0,


In [78]:
#modelRidge_sum_all[0].to_csv('./res_files/Cross_cog/Ridge_All.csv')
modelRidge_sum_all2[0]

,Estimate,t value,p value,value
_intercept,2.287234,24.6842,0.0,
Age,-0.049336,-384.032,0.0,
Sex,-0.000997,-0.0888,0.929243,
Diabetes,0.007378,0.5371,0.591217,
Education,0.116229,9.3931,0.0,
Live Alone,0.156055,11.9388,0.0,
House Condition,0.050493,23.6336,0.0,
Hypertension,0.001827,0.1742,0.861725,
Heart Disease,-0.026975,-1.8787,0.060296,
Alcohol consumption,0.033135,3.3457,0.000822,


### Brazil

In [79]:
#modelRidge_sum_all[0].to_csv('./res_files/Cross_cog/Ridge_All.csv')
modelRidge_sum_br[0]

,Estimate,t value,p value,value
_intercept,0.766141,7.5329,0.0,
Age,-0.021432,-131.6803,0.0,
Sex,0.011312,0.7954,0.42643,
Diabetes,0.011521,0.6365,0.524468,
Education,0.321097,26.0565,0.0,
Live Alone,-0.037272,-1.9179,0.055155,
House Condition,0.028304,12.2726,0.0,
Hypertension,0.005474,0.3917,0.69532,
Heart Disease,-0.01596,-0.741,0.458735,
Alcohol consumption,0.028245,1.2046,0.228411,


### MSE and RMSE

In [80]:
mse_rmse_dict = {}

mse_rmse_dict['MSE_HIC'] = modelRidge_sum_chile_uruguay[4]
mse_rmse_dict['MSE_LMIC'] = modelRidge_sum_ecuador_colombia[4]
mse_rmse_dict['MSE_LMIC_withBR'] = modelRidge_sum_ecu_col_br[4]
mse_rmse_dict['MSE_ALL'] = modelRidge_sum_all[4]
mse_rmse_dict['MSE_ALL2'] = modelRidge_sum_all2[4]
mse_rmse_dict['MSE_BR'] = modelRidge_sum_br[4]


mse_rmse_pd = pd.DataFrame(mse_rmse_dict, index = ['MSE']).T

mse_rmse_pd['RMSE'] = [modelRidge_sum_chile_uruguay[5],
                       modelRidge_sum_ecuador_colombia[5],
                       modelRidge_sum_ecu_col_br[5],
                       modelRidge_sum_all[5],
                       modelRidge_sum_all2[5],
                       modelRidge_sum_br[5]]

mse_rmse_pd

,MSE,RMSE
MSE_HIC,0.726603,0.852410
MSE_LMIC,0.730609,0.854757
MSE_LMIC_withBR,0.663854,0.814773
MSE_ALL,0.672231,0.819897
MSE_ALL2,0.732341,0.855769
MSE_BR,0.378219,0.614995
